In [5]:
import requests
from google.transit import gtfs_realtime_pb2
import pandas as pd
import folium

In [7]:
#step1-setup the api
API_Key="fXNyCp1nmQRY1aX5Ta1SW6rxlfiCX542"
API_URL="https://otd.delhi.gov.in/api/realtime/VehiclePositions.pb?key=fXNyCp1nmQRY1aX5Ta1SW6rxlfiCX542"


In [9]:
#step2-fetching of data(GTFS_real time data)
response=requests.get(API_URL)
if response.status_code !=200:
    raise Exception(f"Error! {response.status_code}:{response.text}")

In [10]:
#step3-data parsing to protbut (data parsing depend on dataset)
feed=gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(response.content)

223829

In [12]:
from datetime import datetime
#step 4=Convert into dataFrame
data=[]
for entity in feed.entity:
    if entity.HasField('vehicle'):
        vehicle=entity.vehicle
    data.append({
        "VehicleID":vehicle.vehicle.id,
        "TripID":vehicle.trip.trip_id,
        "RouteID":vehicle.trip.route_id,
        "Latitude":vehicle.position.latitude,
        "Longitude":vehicle.position.longitude,
        "Timestamp":datetime.fromtimestamp(vehicle.timestamp) if vehicle.timestamp else None
    })
    

In [14]:
dataset=pd.DataFrame(data)
dataset.info()
"Information of captured data"

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2185 entries, 0 to 2184
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   VehicleID  2185 non-null   object        
 1   TripID     2185 non-null   object        
 2   RouteID    2185 non-null   object        
 3   Latitude   2185 non-null   float64       
 4   Longitude  2185 non-null   float64       
 5   Timestamp  2185 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 102.5+ KB


'Information of captured data'

In [19]:
# step 5-map visulization
m=folium.Map(location=[28.613939, 77.209021],zoom_start=11)

for _,row in dataset.iterrows():
    folium.Marker(
        location=[row["Latitude"],row["Longitude"]],
        popup=f"Route:{row['RouteID']}<br>Vehicle:{row['VehicleID']}<br>Time:{row['Timestamp']}",
        icon=folium.Icon(color="blue",icon="bus")
    ).add_to(m)

m.save("Delhi transport mapping.html")
"map saves succesfully"

        

'map saves succesfully'